In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
df = pd.read_csv(file_path + 'Oanda_Aud_Usd_M15_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-01-03 22:00:00   0.76985   0.76999  0.76906    0.76984   0.77106   
1 2021-01-03 22:15:00   0.76996   0.76996  0.76927    0.76945   0.77062   
2 2021-01-03 22:30:00   0.76939   0.76939  0.76909    0.76919   0.77010   
3 2021-01-03 22:45:00   0.76932   0.76951  0.76899    0.76925   0.77019   
4 2021-01-03 23:00:00   0.76942   0.77039  0.76860    0.77035   0.76991   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   0.77125  0.77054    0.77088   0.77046   0.77060  0.76981    0.77036   
1   0.77065  0.77020    0.77020   0.77029   0.77029  0.76982    0.76982   
2   0.77015  0.76980    0.77015   0.76974   0.76974  0.76946    0.76967   
3   0.77101  0.76984    0.76997   0.76976   0.77026  0.76960    0.76961   
4   0.77125  0.76935    0.77057   0.76966   0.77056  0.76915    0.77046   

   Volume  
0      22  
1       9  
2      13  
3      23  
4     360  
--------------------------

In [5]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

In [6]:
# Add technical indicators (for additional features)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=25).mean()

df['atr'] = atr(df)
df['rsi'] = rsi(df)
df['rsi_sma'] = df['rsi'].rolling(50).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['n_macd'], df['n_macdsignal'] = n_macd(df['macd'], df['macdsignal'])
df['slowk'], df['slowd'] = stoch(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])

df['sar'] = psar(df)

df['chop14'] = chop(df)
df['chop36'] = chop(df, lookback=36)

df['vo'] = vo(df['Volume'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-01-04 13:45:00   0.77227   0.77258  0.77159    0.77174   0.77244   
1     2021-01-04 14:00:00   0.77177   0.77220  0.77123    0.77157   0.77192   
2     2021-01-04 14:15:00   0.77152   0.77177  0.77096    0.77164   0.77166   
3     2021-01-04 14:30:00   0.77160   0.77179  0.77073    0.77130   0.77176   
4     2021-01-04 14:45:00   0.77134   0.77174  0.77031    0.77046   0.77147   
...                   ...       ...       ...      ...        ...       ...   
24846 2021-12-31 20:45:00   0.72716   0.72730  0.72654    0.72654   0.72732   
24847 2021-12-31 21:00:00   0.72658   0.72707  0.72658    0.72703   0.72673   
24848 2021-12-31 21:15:00   0.72700   0.72729  0.72693    0.72717   0.72715   
24849 2021-12-31 21:30:00   0.72712   0.72712  0.72663    0.72673   0.72731   
24850 2021-12-31 21:45:00   0.72676   0.72722  0.72661    0.72672   0.72696   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  ... 

In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [8]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'JPY' else pips_to_risk * 100

    if second == 'USD':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'JPY' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, use_chop, chop_cutoff, use_vo, vo_cutoff, use_impact, impact_cutoff, use_n_macd):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = 12
    spread_cutoff /= 10000

    for i in range(lookback, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        ema200_2, ema100_2, ema50_2, ema25_2, atr2, sar2, rsi2, rsi_sma2, mid_open2, mid_close2, mid_low2, mid_high2, vo2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'rsi_sma', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High', 'vo']]
        ema200_1, ema100_1, ema50_1, ema25_1, atr1, sar1, rsi1, rsi_sma1, mid_open1, mid_close1, mid_low1, mid_high1, vo1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'rsi_sma', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High', 'vo']]
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]
        prev_bid_open, prev_ask_open = df.loc[df.index[i - 1], ['Bid_Open', 'Ask_Open']]
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_mid_open = \
            df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                'Ask_Low', 'Ask_Close', 'Mid_Open']]
        chop2 = df.loc[df.index[i - 2], 'chop14']
        chop1 = df.loc[df.index[i - 1], 'chop14']

        emas_buy_signal = ema200_1 < mid_close1
        emas_sell_signal = ema200_1 > mid_close1

        sar_buy_signal = sar1 < min([mid_open1, mid_close1])
        sar_sell_signal = sar1 > max([mid_open1, mid_close1])

        rsi_buy_signal = rsi2 < 50 and rsi1 > 50
        rsi_sell_signal = rsi2 > 50 and rsi1 < 50

        chop_signal = max([chop2, chop1]) < 0.50
        # chop_signal = chop1 < 0.50

        vo_signal = min([vo2, vo1]) > 0.0
        # vo_signal = vo1 > 0.0

        stoch_vals = list(df.loc[df.index[i - 6:i], 'slowk_rsi'])
        stoch_vals_2 = list(df.loc[df.index[i - 6:i], 'slowd_rsi'])
        stoch_buy_signal = False
        stoch_sell_signal = False

        for z in range(len(stoch_vals) - 1, -1, -1):
            if stoch_vals[z] < 20 and stoch_vals_2[z] < 20:
                stoch_buy_signal = True
                break

            elif stoch_vals[z] > 80 or stoch_vals_2[z] > 80:
                stoch_buy_signal = False
                break

        for z in range(len(stoch_vals) - 1, -1, -1):
            if stoch_vals[z] > 80 and stoch_vals_2[z] > 80:
                stoch_sell_signal = True
                break

            elif stoch_vals[z] < 20 or stoch_vals_2[z] < 20:
                stoch_sell_signal = False
                break

        if trade is None:
            if emas_buy_signal and sar_buy_signal and rsi_buy_signal and enough_volatility and chop_signal and vo_signal and stoch_buy_signal:
                if curr_ask_low <= prev_ask_open <= curr_ask_high:
                    open_price = float(prev_ask_open)
                    pullback = sar1

                    stop_loss = round(pullback, 5)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)

                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, 'AUD_USD')

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

            elif emas_sell_signal and sar_sell_signal and rsi_sell_signal and enough_volatility and chop_signal and vo_signal and stoch_sell_signal:
                if curr_bid_low <= prev_bid_open <= curr_bid_high:
                    open_price = float(prev_bid_open)
                    pullback = sar1

                    stop_loss = round(pullback, 5)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)

                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, 'AUD_USD')

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [9]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5, 2]
# risk_reward_ratio_vals = [1.5]
spread_cutoffs = [2.0, 2.5]
# use_chop_vals = [True, False]
use_chop_vals = [False]
# chop_cutoff_vals = [0.50, 0.55, 0.60]
chop_cutoff_vals = [0.50]
# use_vo_vals = [True, False]
use_vo_vals = [True]
# vo_cutoff_vals = [0.0, 0.05, 0.10]
vo_cutoff_vals = [0.0]
# use_impact_vals = [True, False]
use_impact_vals = [False]
# impact_cutoff_vals = [1.0, 1.5, 2.0]
impact_cutoff_vals = [1.0]
# use_n_macd_vals = [True, False]
use_n_macd_vals = [False]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(use_chop_vals) * len(chop_cutoff_vals) * len(use_vo_vals) * len(vo_cutoff_vals) * len(use_impact_vals) * len(impact_cutoff_vals) * len(use_n_macd_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for use_chop in use_chop_vals:
            for chop_cutoff in chop_cutoff_vals:
                for use_vo in use_vo_vals:
                    for vo_cutoff in vo_cutoff_vals:
                        for use_impact in use_impact_vals:
                            for impact_cutoff in impact_cutoff_vals:
                                for use_n_macd in use_n_macd_vals:
                                    all_combos.append((risk_reward_ratio, spread_val, use_chop, chop_cutoff, use_vo, vo_cutoff, use_impact, impact_cutoff, use_n_macd))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_use_chop_val = None
best_chop_cutoff_val = None
best_use_vo_val = None
best_vo_cutoff_val = None
best_use_impact_val = None
best_impact_cutoff_val = None
best_use_n_macd = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, use_chop, chop_cutoff, use_vo, vo_cutoff, use_impact, impact_cutoff, use_n_macd in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, use_chop, chop_cutoff, use_vo, vo_cutoff, use_impact, impact_cutoff, use_n_macd)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    # print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'use_chop': use_chop, 'chop_cutoff': chop_cutoff, 'use_vo': use_vo, 'vo_cutoff': vo_cutoff, 'use_impact': use_impact, 'impact_cutoff': impact_cutoff, 'use_n_macd': use_n_macd})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_use_chop_val = use_chop
        best_chop_cutoff_val = chop_cutoff
        best_use_vo_val = use_vo
        best_vo_cutoff_val = vo_cutoff
        best_use_impact_val = use_impact
        best_impact_cutoff_val = impact_cutoff
        best_use_n_macd = use_n_macd

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 4

201.98297999999676
Num buys: 1
Num sells: 7
Num trades: 8
Num wins: 6
Num losses: 2
Win streak: 3
Loss streak: 1
Remaining runs: 3
Best reward so far: 201.98297999999676

7.315050000000156
Num buys: 1
Num sells: 7
Num trades: 8
Num wins: 3
Num losses: 4
Win streak: 1
Loss streak: 2
Remaining runs: 2
Best reward so far: 201.98297999999676

7.315050000000156
Num buys: 1
Num sells: 7
Num trades: 8
Num wins: 3
Num losses: 4
Win streak: 1
Loss streak: 2
Remaining runs: 1
Best reward so far: 201.98297999999676

201.98297999999676
Num buys: 1
Num sells: 7
Num trades: 8
Num wins: 6
Num losses: 2
Win streak: 3
Loss streak: 1
Remaining runs: 0
Best reward so far: 201.98297999999676



In [10]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best chop val: ' + str(best_use_chop_val))
print('Best chop cutoff val: ' + str(best_chop_cutoff_val))
print('Best vo val: ' + str(best_use_vo_val))
print('Best vo cutoff val: ' + str(best_vo_cutoff_val))
print('Best impact val: ' + str(best_use_impact_val))
print('Best impact cutoff val: ' + str(best_impact_cutoff_val))
print('Best n macd val: ' + str(best_use_n_macd))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 201.98297999999676
Best risk/reward ratio: 1.5
Best spread: 2.5
Best chop val: False
Best chop cutoff val: 0.5
Best vo val: True
Best vo cutoff val: 0.0
Best impact val: False
Best impact cutoff val: 1.0
Best n macd val: False
-----------------------
Top results:
{'reward': 201, 'ratio': 1.5, 'spread': 2.5, 'use_chop': False, 'chop_cutoff': 0.5, 'use_vo': True, 'vo_cutoff': 0.0, 'use_impact': False, 'impact_cutoff': 1.0, 'use_n_macd': False}
{'reward': 7, 'ratio': 2, 'spread': 2.5, 'use_chop': False, 'chop_cutoff': 0.5, 'use_vo': True, 'vo_cutoff': 0.0, 'use_impact': False, 'impact_cutoff': 1.0, 'use_n_macd': False}
{'reward': 7, 'ratio': 2, 'spread': 2.0, 'use_chop': False, 'chop_cutoff': 0.5, 'use_vo': True, 'vo_cutoff': 0.0, 'use_impact': False, 'impact_cutoff': 1.0, 'use_n_macd': False}
{'reward': 201, 'ratio': 1.5, 'spread': 2.0, 'use_chop': False, 'chop_cutoff': 0.5, 'use_vo': True, 'vo_cutoff': 0.0, 'use_impact': False, 'impac